# Inteligência Artificial na Era do E-commerce: Previsão de Vendas Semanais (MLDS)

Este notebook implementa:
- EDA detalhada do dataset `Asset/sales.csv`.
- Preparação/limpeza de dados e engenharia de atributos temporais e lags.
- Validação temporal (TimeSeriesSplit) e métricas (MAE, RMSE, MAPE, R²).
- Treino e comparação de modelos (Linear, Ridge, Lasso, RandomForest, GradientBoosting).
- Seleção do melhor modelo e avaliação em holdout final.
- Persistência do melhor pipeline em `models/best_model.joblib`.
- **Previsão futura**: projeção de vendas para as próximas semanas usando o modelo treinado.

Observação: o dashboard Streamlit está disponível em `streamlit_app.py`.


In [1]:
# Imports e configurações
import os
import sys
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from joblib import dump

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.style.use('seaborn-v0_8-whitegrid')

DATA_PATH = os.path.join('Asset', 'sales.csv')
MODELS_DIR = os.path.join('models')
os.makedirs(MODELS_DIR, exist_ok=True)



ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Carregamento e limpeza básica
df_raw = pd.read_csv(DATA_PATH)

# Padroniza nomes de colunas
df_raw.columns = [c.strip() for c in df_raw.columns]

# Conversões de tipos e ajustes conforme dicionário de dados
df = df_raw.copy()
# Especifica o formato da data: DD-MM-YYYY
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Ajustes de escala
if 'Fuel_Price' in df.columns:
    df['Fuel_Price'] = df['Fuel_Price'] / 1000.0
if 'Unemployment' in df.columns:
    df['Unemployment'] = df['Unemployment'] / 1000.0

# Ordena por data
df = df.sort_values('Date').reset_index(drop=True)

# EDA rápida
display(df.head())
display(df.describe(include='all'))
print('Perdas por coluna:')
print(df.isna().sum())

# Checagem de periodicidade semanal (opcional)
df['week_diff'] = df['Date'].diff().dt.days
print('Resumo dif (dias) entre registros:\n', df['week_diff'].describe())
df = df.drop(columns=['week_diff'])



In [ ]:
# Funções utilitárias: métricas

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))


def mape(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))


def evaluate_regression(y_true, y_pred):
    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': rmse(y_true, y_pred),
        'MAPE': mape(y_true, y_pred),
        'R2': r2_score(y_true, y_pred)
    }



In [ ]:
# Engenharia de atributos para série temporal

def add_time_features(frame: pd.DataFrame) -> pd.DataFrame:
    d = frame.copy()
    d['year'] = d['Date'].dt.year
    d['month'] = d['Date'].dt.month
    d['weekofyear'] = d['Date'].dt.isocalendar().week.astype(int)
    d['quarter'] = d['Date'].dt.quarter
    # Sazonalidade cíclica (semana do ano)
    d['sin_woy'] = np.sin(2 * np.pi * d['weekofyear'] / 52.0)
    d['cos_woy'] = np.cos(2 * np.pi * d['weekofyear'] / 52.0)
    return d


def add_lag_rolling_features(frame: pd.DataFrame, target_col: str = 'Weekly_Sales') -> pd.DataFrame:
    d = frame.copy()
    lags = [1, 2, 3, 4, 52]
    for lag in lags:
        d[f'{target_col}_lag_{lag}'] = d[target_col].shift(lag)
    # Janelas móveis (semana)
    d[f'{target_col}_roll_mean_4'] = d[target_col].shift(1).rolling(window=4, min_periods=2).mean()
    d[f'{target_col}_roll_std_4'] = d[target_col].shift(1).rolling(window=4, min_periods=2).std()
    d[f'{target_col}_roll_mean_12'] = d[target_col].shift(1).rolling(window=12, min_periods=4).mean()
    d[f'{target_col}_roll_std_12'] = d[target_col].shift(1).rolling(window=12, min_periods=4).std()
    return d


def build_features(frame: pd.DataFrame) -> pd.DataFrame:
    d = add_time_features(frame)
    d = add_lag_rolling_features(d, target_col='Weekly_Sales')
    # Ajuste de tipos categóricos/booleanos
    if 'Holiday_Flag' in d.columns:
        d['Holiday_Flag'] = d['Holiday_Flag'].astype(int)
    return d


# Aplica engenharia de atributos
df_feat = build_features(df)

# Remove linhas com NaN geradas por lags/rollings
min_index = df_feat.dropna().index.min()
df_model = df_feat.loc[min_index:].reset_index(drop=True)

print('Shape após features:', df_model.shape)
display(df_model.head())



In [ ]:
# Preparação de variáveis e colunas
TARGET = 'Weekly_Sales'
DATE_COL = 'Date'

feature_cols = [c for c in df_model.columns if c not in [TARGET, DATE_COL]]
num_cols = []
cat_cols = []
for c in feature_cols:
    if pd.api.types.is_numeric_dtype(df_model[c]):
        num_cols.append(c)
    else:
        cat_cols.append(c)

print('Num cols:', len(num_cols))
print('Cat cols:', len(cat_cols))

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

X = df_model[feature_cols]
y = df_model[TARGET]



In [ ]:
# Configuração de validação temporal (TimeSeriesSplit)
N_SPLITS = 5
MIN_TRAIN_SIZE = int(len(df_model) * 0.5)

tscv = TimeSeriesSplit(n_splits=N_SPLITS)

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(random_state=RANDOM_STATE),
    'Lasso': Lasso(random_state=RANDOM_STATE),
    'RandomForest': RandomForestRegressor(n_estimators=400, random_state=RANDOM_STATE, n_jobs=-1),
    'GradientBoosting': GradientBoostingRegressor(random_state=RANDOM_STATE)
}

cv_results = []

for name, model in models.items():
    fold_metrics = []
    for fold_idx, (train_idx, val_idx) in enumerate(tscv.split(X)):
        if train_idx.shape[0] < MIN_TRAIN_SIZE:
            continue
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        pipe = Pipeline(steps=[('pre', preprocessor), ('model', model)])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_val)
        metrics = evaluate_regression(y_val, y_pred)
        metrics['fold'] = fold_idx
        metrics['model'] = name
        fold_metrics.append(metrics)
    if fold_metrics:
        dfm = pd.DataFrame(fold_metrics)
        dfm['model'] = name
        cv_results.append(dfm)

cv_results = pd.concat(cv_results, ignore_index=True)
display(cv_results.groupby('model')[['MAE','RMSE','MAPE','R2']].mean().sort_values('RMSE'))



In [ ]:
# Seleção do melhor modelo por RMSE médio e avaliação em holdout final

summary = cv_results.groupby('model')[['MAE','RMSE','MAPE','R2']].mean().sort_values('RMSE')
best_model_name = summary.index[0]
print('Melhor modelo (CV):', best_model_name)

test_size = int(len(X) * 0.15)
X_train_full, X_test = X.iloc[:-test_size], X.iloc[-test_size:]
y_train_full, y_test = y.iloc[:-test_size], y.iloc[-test_size:]

best_estimator = models[best_model_name]
pipe_best = Pipeline(steps=[('pre', preprocessor), ('model', best_estimator)])
pipe_best.fit(X_train_full, y_train_full)

y_pred_test = pipe_best.predict(X_test)
metrics_test = evaluate_regression(y_test, y_pred_test)
print('Métricas holdout final:', metrics_test)

# Persistência
best_path = os.path.join(MODELS_DIR, 'best_model.joblib')
dump({'pipeline': pipe_best, 'model_name': best_model_name, 'features': feature_cols, 'target': TARGET}, best_path)
print('Modelo salvo em:', best_path)



In [ ]:
# Gráficos: importância de atributos e comparação de verdade vs predito

# Importância: só para modelos baseados em árvore/linear com atributo
importances = None
if hasattr(pipe_best.named_steps['model'], 'feature_importances_'):
    # Para modelos com feature_importances_ (RandomForest, GradientBoosting)
    importances = pipe_best.named_steps['model'].feature_importances_
    
    # Recupera nomes após preprocessamento - precisa ajustar o preprocessor primeiro
    preprocessor_fitted = pipe_best.named_steps['pre']
    
    # Cria um DataFrame de exemplo para obter os nomes das features
    X_sample = X.iloc[:1]  # Uma linha de exemplo
    X_transformed = preprocessor_fitted.transform(X_sample)
    
    # Obtém nomes das features numéricas
    num_names = num_cols
    
    # Obtém nomes das features categóricas após one-hot encoding
    cat_feature_names = []
    if cat_cols:
        try:
            ohe = preprocessor_fitted.named_transformers_['cat'].named_steps['onehot']
            cat_feature_names = list(ohe.get_feature_names_out(cat_cols))
        except:
            # Fallback: usa nomes originais se não conseguir obter os nomes transformados
            cat_feature_names = cat_cols
    
    feature_names = num_names + cat_feature_names
    
    # Cria DataFrame com importâncias
    imp_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False).head(20)
    
    plt.figure(figsize=(10,6))
    sns.barplot(data=imp_df, x='importance', y='feature', color='#4C72B0')
    plt.title('Top 20 Importâncias de Atributos')
    plt.tight_layout()
    plt.show()
    
elif hasattr(pipe_best.named_steps['model'], 'coef_') and hasattr(pipe_best.named_steps['model'], 'intercept_'):
    # Para modelos lineares, coeficientes após padronização (interpretação relativa)
    coefs = pipe_best.named_steps['model'].coef_
    
    # Recupera nomes após preprocessamento
    preprocessor_fitted = pipe_best.named_steps['pre']
    
    # Obtém nomes das features numéricas
    num_names = num_cols
    
    # Obtém nomes das features categóricas após one-hot encoding
    cat_feature_names = []
    if cat_cols:
        try:
            ohe = preprocessor_fitted.named_transformers_['cat'].named_steps['onehot']
            cat_feature_names = list(ohe.get_feature_names_out(cat_cols))
        except:
            # Fallback: usa nomes originais se não conseguir obter os nomes transformados
            cat_feature_names = cat_cols
    
    feature_names = num_names + cat_feature_names
    
    coef_df = pd.DataFrame({'feature': feature_names, 'coef': coefs}).sort_values('coef', ascending=False)
    
    plt.figure(figsize=(10,6))
    sns.barplot(data=coef_df.head(20), x='coef', y='feature', color='#55A868')
    plt.title('Top 20 Coeficientes (escala padronizada)')
    plt.tight_layout()
    plt.show()

# Verdade vs predito no holdout final
plt.figure(figsize=(10,5))
plt.plot(df_model[DATE_COL].iloc[-len(y_test):], y_test.values, label='Verdade', linewidth=2)
plt.plot(df_model[DATE_COL].iloc[-len(y_test):], y_pred_test, label='Predito', linewidth=2)
plt.xticks(rotation=45)
plt.legend()
plt.title('Comparação Verdade vs Predito (Holdout)')
plt.tight_layout()
plt.show()



In [ ]:
# Previsão Futura: Projeção de vendas para as próximas semanas

def forecast_future(df_historical: pd.DataFrame, pipe, feature_cols: list, n_weeks: int = 12) -> pd.DataFrame:
    """Gera previsões futuras usando o modelo treinado."""
    predictions = []
    df_working = df_historical.copy()
    
    # Gera datas futuras
    last_date = df_historical['Date'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=7), periods=n_weeks, freq='W')
    
    # Prepara última linha como base
    last_row = df_historical.iloc[-1]
    recent_mean = df_historical.iloc[-12:].mean()
    
    for i in range(n_weeks):
        # Cria DataFrame temporário com histórico + previsões anteriores + nova semana
        future_row_dict = {'Date': future_dates[i]}
        
        # Features externas (usa médias recentes)
        for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Holiday_Flag']:
            if col in df_historical.columns:
                future_row_dict[col] = recent_mean.get(col, last_row.get(col, 0))
        
        # Adiciona previsão temporária (0) para calcular lags/rolling
        future_row_dict['Weekly_Sales'] = 0
        
        # Converte para DataFrame
        future_row_df = pd.DataFrame([future_row_dict])
        future_row_df = add_time_features(future_row_df)
        
        # Concatena com histórico + previsões anteriores para calcular lags/rolling
        df_temp = pd.concat([df_working, future_row_df], ignore_index=True)
        df_temp = add_lag_rolling_features(df_temp, target_col='Weekly_Sales')
        
        # Extrai apenas a última linha (semana futura) com todas as features
        future_row = df_temp.iloc[-1:].copy()
        
        # Atualiza lags com previsões anteriores
        for lag in [1, 2, 3, 4, 52]:
            lag_col = f'Weekly_Sales_lag_{lag}'
            if lag_col in future_row.columns:
                if i >= lag:
                    # Usa previsão anterior
                    future_row[lag_col] = predictions[i - lag]
                else:
                    # Usa valor histórico
                    hist_idx = len(df_historical) - lag + i
                    if hist_idx >= 0 and hist_idx < len(df_historical):
                        future_row[lag_col] = df_historical['Weekly_Sales'].iloc[hist_idx]
        
        # Prepara features para predição (garante que todas as colunas necessárias estão presentes)
        X_future = future_row[feature_cols].copy()
        
        # Previsão
        pred = pipe.predict(X_future)[0]
        predictions.append(pred)
        
        # Atualiza DataFrame de trabalho para próximos lags
        future_row['Weekly_Sales'] = pred
        df_working = pd.concat([df_working, future_row], ignore_index=True)
    
    # Cria DataFrame final com previsões
    df_forecast = pd.DataFrame({
        'Date': future_dates,
        'Weekly_Sales_Predicted': predictions
    })
    
    # Adiciona features externas para referência
    for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Holiday_Flag']:
        if col in df_historical.columns:
            df_forecast[col] = recent_mean.get(col, last_row.get(col, 0))
    
    return df_forecast


# Gera previsões para as próximas 12 semanas
n_weeks_forecast = 12
df_forecast = forecast_future(df_model, pipe_best, feature_cols, n_weeks=n_weeks_forecast)

print(f'Previsões geradas para as próximas {n_weeks_forecast} semanas:')
display(df_forecast[['Date', 'Weekly_Sales_Predicted']].head(10))

# Gráfico: histórico + previsão
fig_forecast, ax_forecast = plt.subplots(figsize=(12, 5))

# Histórico (últimas 26 semanas)
hist_window = min(26, len(df_model))
hist_dates = df_model[DATE_COL].iloc[-hist_window:]
hist_sales = df_model[TARGET].iloc[-hist_window:]

ax_forecast.plot(hist_dates, hist_sales.values, label='Histórico (últimas 26 semanas)', 
                color='#4C72B0', linewidth=2, alpha=0.7)
ax_forecast.plot(df_forecast['Date'], df_forecast['Weekly_Sales_Predicted'], 
                label=f'Previsão ({n_weeks_forecast} semanas)', 
                color='#55A868', linewidth=2, linestyle='--', marker='o', markersize=4)

# Conecta histórico com previsão
if len(hist_dates) > 0:
    last_hist_date = hist_dates.iloc[-1]
    last_hist_value = hist_sales.iloc[-1]
    first_forecast_date = df_forecast['Date'].iloc[0]
    first_forecast_value = df_forecast['Weekly_Sales_Predicted'].iloc[0]
    
    ax_forecast.plot([last_hist_date, first_forecast_date], 
                   [last_hist_value, first_forecast_value], 
                   color='#55A868', linestyle='--', alpha=0.5)

ax_forecast.axvline(x=hist_dates.iloc[-1] if len(hist_dates) > 0 else df_forecast['Date'].iloc[0], 
                   color='red', linestyle=':', alpha=0.5, label='Hoje')
ax_forecast.set_xlabel('Data')
ax_forecast.set_ylabel('Vendas Semanais')
ax_forecast.set_title(f'Previsão de Vendas - Próximas {n_weeks_forecast} semanas')
ax_forecast.legend()
ax_forecast.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Métricas agregadas da previsão
print('\nMétricas da Previsão Futura:')
print(f"Média Prevista: R$ {df_forecast['Weekly_Sales_Predicted'].mean():,.0f}")
print(f"Pico Previsto: R$ {df_forecast['Weekly_Sales_Predicted'].max():,.0f}")
print(f"Mínimo Previsto: R$ {df_forecast['Weekly_Sales_Predicted'].min():,.0f}")

print('\nNota: As previsões futuras usam médias recentes para variáveis externas.')
print('Para cenários mais precisos, considere usar projeções macroeconômicas reais.')

